In [ ]:
%load_ext autoreload
%autoreload 2

## Super resolution

In [ ]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from torchvision.models import vgg16_bn
torch.cuda.is_available()

In [ ]:
pipeline = 'tensor_pipeline'

In [ ]:
path = Path(f'./data/crappified/dataset_1/{pipeline}')
if pipeline == 'tensor_pipeline':
    path_hr = path/'orig_tensor'
    path_lr = path/'crap_tensor'
if pipeline == 'image_pipeline':
    path_hr = path/'orig_tensor'
    path_lr = path/'crap_tensor'

In [ ]:
if pipeline == 'tensor_pipeline':
    meta = pd.read_csv('./data/crappified/dataset_1/meta/meta_tensor.csv')
if pipeline == 'image_pipeline':
    meta = pd.read_csv('./data/crappified/dataset_1/meta/meta_spectr.csv')
meta.subset = meta.subset == 'valid'

In [ ]:
bs,size=16,128
arch = models.resnet34

if pipeline == 'tensor_pipeline':
    from crystal_clear.tensor_pipeline import TensorImageImageList, TensorImageList
    src = TensorImageImageList.from_df(meta, path_lr).split_from_df('subset')
    data_stats = torch.load('./data/crappified/dataset_1/tensor_pipeline/data_stats.pkl')
    def get_data(bs,size):
        data = (src.label_from_func(lambda x: path_hr/os.path.basename(x), label_cls=TensorImageList)
               .databunch(bs=bs).normalize(data_stats, do_y=True))
        data.c = 3
        return data

if pipeline == 'image_pipeline':
    from fastai.vision.data import imagenet_stats
    src = ImageImageList.from_df(meta, path_lr).split_from_df('subset')
    def get_data(bs,size):
        data = (src.label_from_func(lambda x: path_hr/os.path.basename(x), label_cls=ImageList)
               .databunch(bs=bs).normalize(imagenet_stats, do_y=True))
        data.c = 3
        return data

In [ ]:
data = get_data(bs,size)

## Feature loss

In [ ]:
t = data.valid_ds[0][1].data
t = torch.stack([t,t])

In [ ]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
gram_matrix(t)

In [ ]:
base_loss = F.l1_loss

In [ ]:
vgg_m = vgg16_bn(True).features
model_clas_name = 'model_2_fastai'
new_weights = torch.load(path / f'/{pipeline}/model_clas/{model_clas_name}_weights.pth')
vgg_m.load_state_dict(new_weights, strict= False)
vgg_m.cuda().eval()
requires_grad(vgg_m, False)

In [ ]:
blocks = [i-1 for i,o in enumerate(children(vgg_m)) if isinstance(o,nn.MaxPool2d)]
blocks, [vgg_m[i] for i in blocks]

In [ ]:
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [ ]:
feat_loss = FeatureLoss(vgg_m, blocks[2:5], [5,15,2])

## Train

In [ ]:
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, callback_fns=LossMetrics,
                     blur=True, norm_type=NormType.Weight, path = path / f'{pipeline}')
gc.collect();

In [ ]:
#learn.lr_find()
#learn.recorder.plot()

In [ ]:
lr = 1e-3

In [ ]:
def do_fit(save_name, lrs=slice(lr), pct_start=0.9):
    learn.fit_one_cycle(3, lrs)
    learn.save(save_name)
    learn.show_results(rows=1, imgsize=5)

In [ ]:
do_fit('1a_tensor')

In [ ]:
learn.export('./models_export/1a_tensor.pkl')

In [ ]:
learn.unfreeze()

In [ ]:
#learn.lr_find()
#learn.recorder.plot()

In [ ]:
lr = 1e-3

In [ ]:
do_fit('1b', slice(1e-5,lr))

In [ ]:
learn.export("./models_export/1b.pkl")